<b>Detection of Sargassum on the coast and coastal waters</b>  
Notebook for classifying and analyzing Sargassum in Bonaire with Sentinel-2 images

* Decision Tree Classifier (DTC) and Maximum Likelihood Classifier (MLC) are employed
* Training sites covering 8 different classes are used to extract pixel values (training samples) over all Sentinel-2 bands
* 12 Sentinel bands and 8 spectral indices evaluated using Jeffries-Matusita distance (selected: NDVI, REP, B05 and B11) 
* 80:20 train-test ratio for splitting the training samples
* K-Fold cross-validation performed for tuning the DTC model
* MLC model developed with 4 different chi-square thresholds: 0% (base), 10%,20%,50%


In [ ]:
import os
import re
import pandas as pd
import numpy as np
import rasterio as rio
from rasterio import Affine
from rasterio.mask import mask
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import geopandas as gpd
from joblib import dump,load
from rasterstats import zonal_stats
from tqdm import tqdm,tqdm_notebook

#custom functions
from Python.prep_raster import stack_bands,clip_raster,pixel_sample,computeIndexStack
from Python.data_treat import balance_sample,down_sample
from Python.spec_analysis import transpose_df,jmd2df
from Python.data_viz import specsign_plot,jmd_heatmap,ridgePlot,validation_curve_plot
from Python.mlc import mlClassifier
from Python.calc_acc import calc_acc
from Python.pred_raster import stack2pred, dtc_pred_stack
from Python.misc import get_feat_layer_order

#sklearn functions
from sklearn.model_selection import train_test_split,validation_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier

#setup IO directories
parent_dir = os.path.join(os.path.abspath('..'),'objective1')                  #change according to preference
sub_dirs = ['fullstack','clippedstack','indexstack','predicted','stack2pred']
make_dirs = [os.makedirs(os.path.join(parent_dir,name),exist_ok=True) for name in sub_dirs]

<b>Sentinel-2 data preparation</b>
* Resample coarse bands to 10m resolution
* Stack multiband images 
* Calculate spectral indices

In [ ]:
#dates considered for classification and analysis 
dates = [20180304,20180309,20180314,20180319,20190108,20190128,20190212,20190304,20190309, 
         20190314,20190319,20190508,20190513,20190518,20190523,20190821,20191129]

#band names
bands = ['B01_60m','B02_10m','B03_10m','B04_10m','B05_20m','B06_20m',
         'B07_20m','B08_10m','B8A_20m','B09_60m','B11_20m','B12_20m']

#get product file paths according to dates and tile ID T19PEP (covers Bonaire)
level2_dir = '...' #change according to preference
level2_files = glob(level2_dir+"/*.SAFE")
scene_paths=[file for date in dates for file in level2_files if str(date) in file and 'T19PEP' in file]

#sort multiband image paths according to date
image_collection ={}

for scene in scene_paths:
    date = re.findall(r"(\d{8})T", scene)[0]
    
    #collect all .jp2 band images in SAFE directory
    all_images = [f for f in glob(scene + "*/**/*.jp2", recursive=True)]
    img_paths = [img_path for band in bands for img_path in all_images if band in img_path]
    image_collection[date] = img_paths

#check nr. of images per date
for key in image_collection.keys():print(f'Date: {key} Images: {len(image_collection[key])}')

In [ ]:
#stack multiband images to a geotiff (!computationaly intensive)
for date in tqdm(image_collection.keys(),position=0, leave=True):
    ref10m= image_collection[date][1]                      #use band B02 (10m) as reference metadata
    outfile = os.path.join(parent_dir,'fullstack',f'stack_{date}.tif')
    stack_bands(image_collection[date],ref10m,outfile)


In [ ]:
#crop multiband image stack and compute spectral indices
roi_file = './data/boundaries/coastline_lacbay.geojson'                   #polygon for cropping image
indices = ['NDVI','REP','FAI','GNDVI','NDVI_B8A','VB_FAH','SEI','SABI']   #list of indices used in the study

stack_files = glob(parent_dir + "/fullstack/*.tif")
for stack_file in tqdm(stack_files,position=0, leave=True):
    filename = os.path.basename(stack_file).split('.')[0]
    
    #cropping
    clip_outfile = os.path.join(parent_dir,'clippedstack',filename+"_clipped.tif")
    clip_raster(stack_file,roi_file,clip_outfile,fill=True,nodat=0)
    
    #compute spectral indices
    index_outfile = os.path.join(index_dir,filename+"_index.tif")
    computeIndexStack(clip_outfile,indices,index_outfile)
    

<b>Sample pixel values from multiband images based on training sites</b>  
* Training scenes from 4,9,14 and 19 March 2019

In [ ]:
#get training sites and corresponding images
train_sites = [f for f in glob(r".\data\training_input\objective1\*_coast.geojson")]     
dates = [20190304,20190309,20190314,20190319]                                 
stack_bands = [f for date in dates for f in glob(parent_dir+'/clipped*/*_clipped.tif') if str(date) in f]   
index_bands = [f for date in dates for f in glob(parent_dir+'/index*/*_index.tif') if str(date) in f] 

#bands and indices to be sampled
band_names = ['B01','B02','B03','B04','B05','B06','B07','B08','B8A','B09','B11','B12']
indices = ['NDVI','REP','FAI','GNDVI','NDVI-B8A','VB-FAH','SEI','SABI']

dataset = []
for i in range(len(train_sites)):
    
    #sample multibands and spectral indices
    df_bands= pixel_sample(stack_bands[i],train_sites[i],band_names)
    df_indices= pixel_sample(index_bands[i],train_sites[i],indices)
    df_sample = pd.concat([df_bands,df_indices],axis=1)
    df_sample = df_sample.loc[:,~df_sample.columns.duplicated()]
    
    #downsample based on floating Sargassum (Sf)
    df_downsampled = down_sample(df_sample,'C','Sf')
    dataset.append(df_downsampled)
    
#final dataset
dataset=pd.concat(dataset,sort=False).reset_index(drop=True) 
dataset.to_csv(r'./data/training_input/csv/training_samples_20190304_20190319_sargassum.csv',index=False)

<b>Expore spectral signature</b>  
* Jeffries-Matusita distance (JMD) used for feature selection ([reference](https://books.google.nl/books?id=RxHbb3enITYC&pg=PA52&lpg=PA52&dq=for+one+feature+and+two+classes+the+Bhattacharyya+distance+is+given+by&source=bl&ots=sTKLGl1POo&sig=ACfU3U2s7tv0LT9vfSUat98l4L9_dyUgeg&hl=nl&sa=X&ved=2ahUKEwiKgeHYwI7lAhWIIlAKHZfJAC0Q6AEwBnoECAkQAQ#v=onepage&q&f=false))
* NDVI, REP, B05 and B11 are selected as input features for the classifiers

In [ ]:
#load training sample
df = pd.read_csv('./data/training_input/csv/training_samples_20190304_20190319_sargassum.csv')

#plot spectral signature focused on 4 subclasses
specsign_plot(df,df.columns[4:16],classtype='C')

#plot JMD heatmap for each band
jmd_bands = [jmd2df(transpose_df(df,'C',band)) for band in df.columns[4:16]]
jmd_heatmap(jmd_bands)

#plot JMD heatmap for each spectral index
jmd_indices = [jmd2df(transpose_df(df,'C',band)) for band in df.columns[16:]]
jmd_heatmap(jmd_indices)

#plot distribution of selected input features
sns.set_style('white')
ridgePlot(df[['C','NDVI','REP','B05','B11']],'C')

<b>Build classifiers</b>  

In [ ]:
#load training sample
df = pd.read_csv('./data/training_input/csv/training_samples_20190304_20190319_sargassum.csv')
predictors = ['NDVI','REP','B05','B11']
subset_df = df[['C']+predictors]

#split into train and test datasets 80:20
train,test = train_test_split(subset_df, train_size = 0.8,random_state=1,shuffle=True,stratify=np.array(subset_df['C']))
train = train.sort_values(by='C',ascending=True) #sort labels

#split pedictors from labels (for DTC)
le = LabelEncoder()
X_train,y_train = train[predictors],le.fit_transform(train['C'])
X_test,y_test = test[predictors],le.fit_transform(test['C'])

* Decision Tree Classifier

In [ ]:
#perform k-fold (=10) cross-validation 

#parameters considered in this step
max_depth = np.arange(1,40,2)                    
min_samples_split = list(range(2, 100,10))         
max_leaf_nodes= list(range(2, 50,5))              
min_samples_leaf= list(range(1, 100,10))           
min_impurity_decrease=[0,0.00005,0.0001,0.0002,0.0005,0.001,0.0015,0.002,0.005,0.01,0.02,0.05,0.08]   
criterion = ['gini','entropy']

#assign parameters to a dictionary
params = {'max_depth':max_depth,'min_samples_split':min_samples_split,
          'max_leaf_nodes':max_leaf_nodes,'min_samples_leaf':min_samples_leaf,
          'min_impurity_decrease':min_impurity_decrease,'criterion':criterion}

#plot validation curve
fig,axs = plt.subplots(3,2,figsize=(10,8))
axs = axs.ravel()
dtc = DecisionTreeClassifier(random_state=1,criterion='entropy')                    #default model

for (param_name,param_range),i in zip(params.items(),range(len(params.items()))):
    train_scores,test_scores = validation_curve(dtc,X_train.values,y_train,cv=10,scoring='accuracy',
                                                n_jobs=-1,param_range=param_range,param_name=param_name)
    validation_curve_plot(train_scores,test_scores,param_range,param_name,axs[i])
plt.show()

In [ ]:
#train dtc model based on best parameters
dtc = DecisionTreeClassifier(max_depth=5,random_state=2,criterion='entropy',min_samples_split=70,
                             max_leaf_nodes=15,min_samples_leaf=40,min_impurity_decrease=0.01,max_features=4)
dtc = dtc.fit(X_train,y_train)

#export model as joblib file
dump(dtc,r".\data\models\dtc_model_sargassum.joblib")

* Maximum Likelihood Classifier

In [ ]:
#train mlc model
mlc = mlClassifier(train,'C')

#export model as joblib file
dump(mlc,r".\data\models\mlc_model_sargassum.joblib")

* Compute model accuracies (based on test split)

In [ ]:
#load models
dtc = load(r".\data\models\dtc_model_sargassum.joblib")
mlc = load(r".\data\models\mlc_model_sargassum.joblib")

#DTC model accuracy
dtc_y_pred = dtc.predict(X_test)
con_mat_dtc = calc_acc(le.inverse_transform(y_test),le.inverse_transform(dtc_y_pred))
con_mat_dtc['classifier'] = 'DTC'

#MLC model accuracies with chi-square threshold
chi_table = {'MLC base':None,'MLC 10%':7.78,'MLC 20%':5.99,'MLC 50%':3.36}

mlc_conmats = []
for key,value in chi_table.items():
    con_mat_mlc = mlc.classify_testdata(test,'C',threshold=value)
    con_mat_mlc['classifier'] = key
    mlc_conmats.append(con_mat_mlc)

#export model accuracies
mlc_conmats = pd.concat(mlc_conmats)
model_acc = pd.concat([con_mat_dtc,mlc_conmats])
model_acc.to_csv('./data/output/objective1/dtc_mlc_model_acc_obj1.csv')

<b>Classification</b>  
* create an image stack for prediction (stack2pred) for all scenes in objective1 folder
* classify each stack2pred image with the DTC and MLC models

In [ ]:
#get all multiband and spectral index images
stack_bands =  glob(parent_dir+'/clipped*/*_clipped.tif')
index_bands = glob(parent_dir+'/index*/*_index.tif')

#get the order of the selected predictors in the multiband and spectral index images
predictors = ['NDVI','REP','B05','B11']
used_indices, used_bands = get_feat_layer_order(predictors)

stack2pred_paths = []

#create stack2pred rasters
for band_image,index_image in zip(stack_bands,index_bands):
    date = re.findall(r"(\d{8})", band_image)[0]
    outfile = os.path.join(f'{parent_dir}\stack2pred',f'stack2pred_{date}.tif')
    stack2pred_paths.append(outfile)

    stack2pred(index_image,band_image,used_indices,used_bands,outfile)
    

In [ ]:
#load models
dtc = load(r".\data\models\dtc_model_sargassum.joblib")
mlc = load(r".\data\models\mlc_model_sargassum.joblib")

#stack2pred image paths
stack2pred_paths = glob(parent_dir+'*/stack2pred/stack2pred_*.tif')

#classify all stack2pred images
for path in stack2pred_paths:
    
    date = re.findall(r"(\d{8})", path)[0]
    
    #predict multiple mlc with thresholds
    mlc_out = f'{parent_dir}/predicted/mlc/mlc_{date}_multi.tif'
    os.makedirs(os.path.dirname(mlc_out),exist_ok=True)
    if not os.path.exists(mlc_out):
        chi_probs = [None,7.78,5.99,3.36]
        mlc_preds = np.array([mlc.classify_raster_gx(path,out_file=None,threshold=prob) for prob in chi_probs])

    #export multilayer mlc image
    with rio.open(path) as src:
        profile = src.profile.copy()
        profile.update({'dtype': rio.uint16})
        with rio.open(mlc_out ,'w',**profile) as dst:
            dst.write(mlc_preds.astype(rio.uint16))
    
    #predict and export DTC raster
    dtc_out = f'{parent_dir}/predicted/dtc/dtc_{date}.tif'
    os.makedirs(os.path.dirname(dtc_out),exist_ok=True)
    if not os.path.exists(dtc_out):
        dtc_pred_stack(dtc,path,dtc_out)

* MLC class posterior probability raster

In [ ]:
#stack2pred image paths
stack2pred_paths = glob(parent_dir+'*/stack2pred/stack2pred_*.tif')

#compute probabality raster
for path in stack2pred_paths:
    mlc_prob_out = f'{parent_dir}/predicted/mlc/mlc_{date}_prob.tif'
    os.makedirs(os.path.dirname(mlc_out),exist_ok=True)
    mlc.prob_rasters(path,mlc_prob_out)

<b>External validity</b>  
* Classify DTC and MLC results for a scene taken on 2019-05-18
* Validation samples only covers Non-Floating Sargassum (Non-Sf) and Floating Sargassum (Sf)
* Floating Sargassum (Sf) pixel value = 3 in the DTC and MLC rasters 

In [ ]:
#get file paths
val_samples = gpd.read_file(r'./data/training_input/objective1/sf_validation_20190518.geojson')
dtc_file = glob(parent_dir+'/predicted*/dtc/dtc*20190518*.tif')[0]
mlc_file = glob(parent_dir+'/predicted*/mlc/mlc*20190518*.tif')[0]

coords =  [(val_samples.geometry[i][0].x,val_samples.geometry[i][0].y) for i in range(len(val_samples))]
with rio.open(dtc_file) as dtc_src, rio.open(mlc_file) as mlc_src:
    
    #sample from dtc raster
    val_samples['DTC'] = [pt[0] for pt in dtc_src.sample(coords)]
    
    #sample from multilayer mlc raster
    mlc_multi = pd.concat([pd.DataFrame(pt).T for pt in mlc_src.sample(coords)],ignore_index=True)
    val_samples[['MLC base','MLC 10%','MLC 20%','MLC 50%']] = mlc_multi
    
#convert pixel values to 1 if Sf, else to 0 for others
val_samples[val_samples.columns[-5:]] = (val_samples[val_samples.columns[-5:]]==3).astype(int)

#compute classification (validation) accuracy 
df_val = pd.DataFrame(val_samples.drop(columns='geometry'))

acc_val_dfs = []
for pred in df_val.columns[df_val.columns!='label']:
    acc = calc_acc(df_val['label'].values, df_val[pred].values)
    acc['classifier'] = pred
    acc_val_dfs.append(acc)
acc_val_dfs = pd.concat(acc_val_dfs)
acc_val_dfs.to_csv('./data/output/objective1/dtc_mlc_external_val_obj1.csv')

* Plot model and validation accuracies

In [ ]:
model_df = pd.read_csv('./data/output/objective1/dtc_mlc_model_acc_obj1.csv').set_index('Model')
val_df  = pd.read_csv('./data/output/objective1/dtc_mlc_external_val_obj1.csv').set_index('Observed')

acc2plot = {'Model accuracy (8 classes)':model_df.loc['PA','UA'].str[:4].astype(float),
            'Model F1-score (Sf)':model_df.loc['Sf','F1-score'].astype(float),
            'Validation accuracy (2 classes)':val_df.loc['PA','UA'].str[:4].astype(float),
            'Validation F1-score (Sf)':val_df.loc['1','F1-score'].astype(float)}

[plt.plot(val_df['classifier'].unique(),value,label=key) for key,value in acc2plot.items()]
plt.legend()

<b>Comparative analysis</b>  
* Compare Sargassum (Sf and Sl) classified area across different scenes for each model
* Persisting missclassification occur between the two Sargassum classes and other coastal features, hence a mask was applied.

In [ ]:
#get classification result paths
dtc_paths = glob(parent_dir+'/predicted*/dtc/dtc*.tif')
mlc_paths = glob(parent_dir+'/predicted*/mlc/mlc*.tif')

#load mask
sl_mask = [gpd.read_file('./data/boundaries/sf_sl_mask.geojson').__geo_interface__['features'][0]['geometry']]
sf_mask = [gpd.read_file('./data/boundaries/sf_sl_mask.geojson').__geo_interface__['features'][1]['geometry']]

#collection of Sargassum classification results
data = dict.fromkeys(['Date','Sl MLC Base','Sl MLC 10%','Sl MLC 20%','Sl MLC 50%','Sl DTC',
                     'Sf MLC Base','Sf MLC 10%','Sf MLC 20%','Sf MLC 50%','Sf DTC'], [])

for i in range(len(mlc_paths)):
    date = re.findall(r"(\d{8})", mlc_paths[i])
    data['Date'] = data['Date']+ [str(pd.to_datetime(date)[0].date())]
    
    with rio.open(dtc_paths[i]) as dtc_src, rio.open(mlc_paths[i]) as mlc_src:
        
        #sf pixel count
        dtc_img= mask(dataset=dtc_src,shapes=sf_mask,nodata=dtc_src.nodata,invert=True)[0]
        data['Sf DTC'] = data['Sf DTC']+[np.unique(dtc_img, return_counts=True)[1][2]]
        
        mlc_imgs= mask(dataset=mlc_src,shapes=sf_mask,nodata=mlc_src.nodata,invert=True)[0]
        for k,sf_mlc_key in enumerate(list(data.keys())[6:-1]): 
            data[sf_mlc_key] = data[sf_mlc_key]+ [[np.unique(mlc_img, return_counts=True)[1][2] for mlc_img in mlc_imgs][k]]
        
        #sl pixel count
        dtc_img= mask(dataset=dtc_src,shapes=sl_mask,nodata=dtc_src.nodata,invert=False)[0]
        data['Sl DTC'] = data['Sl DTC']+[np.unique(dtc_img, return_counts=True)[1][3]]
        
        mlc_imgs= mask(dataset=mlc_src,shapes=sl_mask,nodata=mlc_src.nodata,invert=False)[0]
        for j,sl_mlc_key in enumerate(list(data.keys())[1:5]): 
            data[sl_mlc_key] = data[sl_mlc_key]+[[np.unique(mlc_img, return_counts=True)[1][3] for mlc_img in mlc_imgs][j]]

#export data
data = pd.DataFrame(data)
data.to_csv('./data/output/objective1/classified_area_obj1.csv',index=False)

* Plot Sargassum classified area in 2019

In [ ]:
#load data and subset only the 2019 results
data = pd.read_csv('./data/output/objective1/classified_area_obj1.csv',index_col='Date')[4:]

#plot Floating Sargassum (Sf) and Sargassum on land (Sl)
fig,axs = plt.subplots(1,2,figsize=(20,8))
axs[0].set_ylabel('Classified area (ha)')
plt.tight_layout()
fig.autofmt_xdate()

plots = [axs[0].plot(data[col]/100) if 'Sf' in col else axs[1].plot(data[col]/100) for col in data.columns]
legends = axs[0].legend(data.columns[:5],loc='upper right'),axs[1].legend(data.columns[5:],loc='upper right')

<b>Sargassum coverage maps</b>  
* Compute Sargassum coverage maps  for the invasions in March and May 2019 and March 2018
* A 20mx20m grid was used to calculate the coverage for each scene
* MLC 20% results were used for Floating Sargassum (Sf) coverage map
* MLC 50% results were used for Sargassum on land (Sl) coverage map
* Note that code below takes about 10 minutes to run (due to small grid tile size)


In [ ]:
#get classification result paths
mlc_paths = glob(parent_dir+'/predicted*/mlc/mlc*03*.tif')+glob(parent_dir+'/predicted*/mlc/mlc*05*.tif')

#load mask and grid data
mask_data = gpd.read_file('./data/boundaries/objective1/sf_sl_mask.geojson').__geo_interface__['features']
grid_file = gpd.read_file(r'./data/boundaries/objective1/20mgrid.geojson')

#collect geodataframes
data = []

for mlc_file in mlc_paths:
    date = re.findall(r"(\d{8})", mlc_file)[0]
    with rio.open(mlc_file) as src:
        
        #iterate according to mask data (first item = sl, second item = sf)
        #count number of pixel in each grid tile (computationaly intensive!)
        for feat,label,val,inv,model in zip(mask_data,['sl','sf'],[4,3],[False,True],[3,2]):
            img = mask(dataset=src,shapes=[feat['geometry']],nodata=src.nodata,invert=inv)[0][model]
            zs = zonal_stats(grid_file,np.where(img==val,1,0),affine=src.transform,
                            prefix=f'{label}_{date}_',stats='count',geojson_out=True,nodata=0)
            zs_filter = list(filter(lambda x: x['properties'][f'{label}_{date}_count']!=0, zs))
            data.append(gpd.GeoDataFrame.from_features(zs_filter,crs=grid_file.crs))

#merge with grid file based on id
grid_file_copy = grid_file.copy()
for i in range(len(data)):
    grid_file_copy = gpd.GeoDataFrame(grid_file_copy.merge(data[i][data[i].columns[1:]],on='id',how='outer'),
                                  crs=grid_file.crs,geometry=grid_file.geometry).replace(np.nan,0)

#calculate coverage for each grid tile 
sf_split = np.array_split(grid_file_copy[[i for i in grid_file_copy.columns if 'sf' in i ]],3,axis=1)
sl_split = np.array_split(grid_file_copy[[i for i in grid_file_copy.columns if 'sl' in i ]],3,axis=1)
scale_factor = (100/4/400) #(relative coverage of Sentinel-2 pixels in a 20x20m tile over 4 dates)
sf_covr = [sf_split[i].sum(1)*scale_factor for i in range(len(sf_split))]
sl_covr = [sl_split[i].sum(1)*scale_factor for i in range(len(sl_split))]

#export coverage maps
gdf_out = pd.concat([grid_file_copy[['geometry']]]+sf_covr+sl_covr,axis=1)
gdf_out.columns = ['geometry','sf_mar2018','sf_mar2019','sf_may2019','sl_mar2018','sl_mar2019','sl_may2019']
gdf_out = gdf_out[gdf_out[gdf_out.columns[1:]].sum(1)!=0]
gdf_out.to_file(r'./data/output/objective1/sargassum_coverage_coast.geojson',driver='GeoJSON')